<a href="https://colab.research.google.com/github/sujiDuddaiyan/GLM/blob/main/GLM_text_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

In [ ]:
#Try reading the file while handling bad lines
df = pd.read_csv("/content/news-article-categories.csv", on_bad_lines='skip')


# Print any rows that were skipped due to errors
print(df)

            category                                              title  \
0     ARTS & CULTURE  Modeling Agencies Enabled Sexual Predators For...   
1     ARTS & CULTURE  Actor Jeff Hiller Talks “Bright Colors And Bol...   
2     ARTS & CULTURE  New Yorker Cover Puts Trump 'In The Hole' Afte...   
3     ARTS & CULTURE  Man Surprises Girlfriend By Drawing Them In Di...   
4     ARTS & CULTURE  This Artist Gives Renaissance-Style Sculptures...   
...              ...                                                ...   
6872           WOMEN  Casually Fearless: Why Millennials Are Natural...   
6873           WOMEN                               Happy Birthday To Us   
6874           WOMEN                                The Culture of Love   
6875           WOMEN                            Carpe Diem, Oprah Style   
6876           WOMEN                    My Month of Online Dating Detox   

                                                   body  
0     In October 2017, Carolyn Kramer rec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess(text):
    if isinstance(text, str):  # Check if the input is a string
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = re.sub(r'\W', ' ', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove single characters
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text)
    return text

# Preprocess all documents
preprocessed_data = df.applymap(preprocess) # Use applymap to apply the function to each element in the DataFrame

# Tokenize into sentences
sentences = [sent_tokenize(doc) for doc in preprocessed_data.astype(str)] # Convert preprocessed data to string type
sentences = [sent for sublist in sentences for sent in sublist]

# Remove stop words
stop_words = set(stopwords.words('english'))
sentences = [' '.join([word for word in sentence.split() if word not in stop_words]) for sentence in sentences]

<ipython-input-9-392a5538da4e>:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preprocessed_data = df.applymap(preprocess) # Use applymap to apply the function to each element in the DataFrame


In [ ]:
# Vectorize sentences using TF-IDF
vectorizer = TfidfVectorizer()
sentence_vectors = vectorizer.fit_transform(sentences)
print("sentence vectorizer")
print(sentence_vectors)
# Compute cosine similarity matrix
cosine_sim_matrix = cosine_similarity(sentence_vectors, sentence_vectors)

# Function to extract summary
def extract_summary(sentences, cosine_sim_matrix, num_sentences=5):
    sentence_scores = cosine_sim_matrix.sum(axis=1)
    ranked_sentences = [sentences[i] for i in np.argsort(sentence_scores, axis=0)[::-1]]
    summary = ' '.join(ranked_sentences[:num_sentences])
    return summary

# Extract summary
summary = extract_summary(sentences, cosine_sim_matrix, num_sentences=5)
print("Extractive Summary:\n", summary)

sentence vectorizer
  (0, 1)	1.0
  (1, 2)	1.0
  (2, 0)	1.0
Extractive Summary:
 body title category


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load pre-trained model and tokenizer
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def abstractive_summary(text, num_sentences=5):
    # Encode the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Prepare the text for summarization
text = ' '.join(sentences)  # Combine the sentences into a single text
summary = abstractive_summary(text)
print("Abstractive Summary:\n", summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Abstractive Summary:
 summarize: category title body. summarize. body: category name body. Summarize the title body: title body, category name, category title.


In [ ]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f2a4ef3ca1bb40afc0d87341fab2ca31cf155a6e0e69385aec89a4975465ac84
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def evaluate_summary(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

# Example usage:
reference_summary = "This is the reference summary. It should be concise and cover all main points."
generated_summary = extract_summary(sentences, cosine_sim_matrix, num_sentences=5)

# For extractive summarization
print("Extractive Summary:\n", generated_summary)
extractive_scores = evaluate_summary(reference_summary, generated_summary)
print("Extractive Summarization ROUGE Scores:\n", extractive_scores)

# For abstractive summarization
text = ' '.join(sentences)
generated_summary_abstractive = abstractive_summary(text)
print("Abstractive Summary:\n", generated_summary_abstractive)
abstractive_scores = evaluate_summary(reference_summary, generated_summary_abstractive)
print("Abstractive Summarization ROUGE Scores:\n", abstractive_scores)

Extractive Summary:
 body title category
Extractive Summarization ROUGE Scores:
 {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
Abstractive Summary:
 summarize: category title body. summarize. body: category name body. Summarize the title body: title body, category name, category title.
Abstractive Summarization ROUGE Scores:
 {'rouge1': Score(precision=0.05263157894736842, recall=0.07142857142857142, fmeasure=0.06060606060606061), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.05263157894736842, recall=0.07142857142857142, fmeasure=0.06060606060606061)}
